<a href="https://colab.research.google.com/github/dipanjanS/training-fine-tuning-large-language-models-workshop-dhs2024/blob/main/Module-01-LLMs-Transformers-Generative-AI-Essentials/Module_01_LC2_Contextual_Embeddings_and_Semantic_Search_Engines_with_Transformers_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contextual Embeddings and Semantic Search Engines with Transformers

# Contextual Embeddings in Transformers

Contextual embeddings are different from plain word embeddings.

Due to aspects like self-attention, each word's corresponding emebdding can be influenced by surrounding words context.

e.g

- They are going to **fire** him
- He is sitting by the **fire**

The word **fire** would have a different embedding vector because of other words when using transformer models

Also you can typically average word embeddings in a document to get an overall document embedding

Let's start by taking a few different documents.

In [ ]:
import numpy as np

In [ ]:
docs = ['The cat is running on the stairs',
        'A woman is eating a piece of bread.',
        'A man is eating a pasta.']

docs

## Load a pre-trained BERT Transformer model

Here we will use a pre-trained transformer model to get contextual word embeddings and average them to get document embeddings.

![](https://i.imgur.com/4uYtfkQ.png)

Then we can compute simple cosine similarity between document embedding for each pair of the above documents

In [ ]:
from transformers import AutoTokenizer, AutoModel

# model details: https://huggingface.co/google-bert/bert-base-uncased
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

## Tokenizing Documents

Transformers have been trained by first tokenizing the documents, hence each model comes with its own trained tokenizer which can break down a document into individual tokens (words) and each token would have a corresponding embedding.

REMEMBER: We are not training any models here, just using an already pre-trained model on a lot of data and getting embeddings for our documents by passing them through the pre-trained model

In [ ]:
docs[0]

In [ ]:
token_ids = tokenizer(docs[0], return_tensors='pt')
token_ids

Key token ID elements above include

#### Input IDs
The input ids are often the only required parameters to be passed to the model as input. They are token indices, numerical representations of tokens building the sequences that will be used as input by the model. __You should mostly care about this in most cases__

<br/>

#### Attention mask
The attention mask is an optional argument used when batching sequences together. This argument indicates to the model which tokens should be attended to, and which should not (masked language modeling)

<br/>

#### Token Type IDs
Some models’ purpose is to do sequence classification or question answering. These require two different sequences to be joined in a single “input_ids” entry, which usually is performed with the help of special tokens, such as the classifier ([CLS]) and separator ([SEP]) tokens. For example, the BERT model builds its two sequence input as such:

```
>>> # [CLS] SEQUENCE_A [SEP] SEQUENCE_B [SEP]
```

https://huggingface.co/transformers/glossary.html#token-type-ids

In [ ]:
token_ids

In [ ]:
[tokenizer.decode(id) for id in token_ids['input_ids'][0]]

In [ ]:
contextual_embeddings = model(**token_ids)[0]
contextual_embeddings

In [ ]:
contextual_embeddings.shape

In [ ]:
import torch

doc_embedding = torch.mean(contextual_embeddings, dim=1)[0]
doc_embedding

In [ ]:
doc_embedding.shape

In [ ]:
docs

In [ ]:
doc_token_ids = [tokenizer(doc, return_tensors='pt') for doc in docs]
doc_token_ids

In [ ]:
doc_contextual_embeddings = [model(**token_ids)[0] for token_ids in doc_token_ids]
doc_contextual_embeddings

In [ ]:
doc_embeddings = [torch.mean(contextual_embeddings, dim=1)[0] for contextual_embeddings in doc_contextual_embeddings]
doc_embeddings

In [ ]:
doc_embeddings = [np.round(doc_embedding.detach().numpy(), 3) for doc_embedding in doc_embeddings]
doc_embeddings

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(doc_embeddings)

In [ ]:
docs

# Finding semantic similarity with Pre-trained Embeddings

Here we will leverage already pre-trained embedding models \ deep learning models to extract embeddings from sentences and find out their semantic similarity.

Models we will look at:

- Transformers

# Semantic Search

Semantic search seeks to improve search accuracy by understanding the content of the search query. In contrast to traditional search engines which only find documents based on lexical matches, semantic search can also find synonyms.


The idea behind semantic search is to embed all entries in your corpus, whether they be sentences, paragraphs, or documents, into a vector space.

At search time, the query is embedded into the same vector space and the closest embeddings from your corpus are found. These entries should have a high semantic overlap with the query.

![](https://i.imgur.com/FyUCkG5.png)

___[Created By: Dipanjan (DJ)](https://www.linkedin.com/in/dipanjans/)___

# Fun with Embeddings: Simple Search Engine!

Let's create a corpus of documents which will be our source on which we will run text searches

In [ ]:
documents = [
  "Quantum mechanics describes the behavior of very small particles.",
  "Photosynthesis is the process by which green plants make food using sunlight.",
  "Shakespeare's plays are a testament to English literature.",
  "Artificial Intelligence aims to create machines that can think and learn.",
  "The pyramids of Egypt are historical monuments that have stood for thousands of years.",
  "Biology is the study of living organisms and their interactions with the environment.",
  "Music therapy can aid in the mental well-being of individuals.",
  "The Milky Way is just one of billions of galaxies in the universe.",
  "Economic theories help understand the distribution of resources in society.",
  "Yoga is an ancient practice that involves physical postures and meditation."
]

## Building Robust Semantic Search Engines with Transformers

In [ ]:
!pip install sentence-transformers

## Load Dependencies

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

## Load Pre-trained Transformer Model

In [ ]:
# https://huggingface.co/microsoft/MiniLM-L12-H384-uncased
# MiniLM: Small and Fast Pre-trained Models for Language Understanding and Generation
# MiniLMv1-L12-H384-uncased: 12-layer, 384-hidden, 12-heads, 33M parameters, 2.7x faster than BERT-Base

model = SentenceTransformer('all-MiniLM-L12-v2')

In [ ]:
model

## Create a corpus of documents

In [ ]:
documents = ['Quantum mechanics describes the behavior of very small particles.',
 'Photosynthesis is the process by which green plants make food using sunlight.',
 "Shakespeare's plays are a testament to English literature.",
 'Artificial Intelligence aims to create machines that can think and learn.',
 'The pyramids of Egypt are historical monuments that have stood for thousands of years.',
 'Biology is the study of living organisms and their interactions with the environment.',
 'Music therapy can aid in the mental well-being of individuals.',
 'The Milky Way is just one of billions of galaxies in the universe.',
 'Economic theories help understand the distribution of resources in society.',
 'Yoga is an ancient practice that involves physical postures and meditation.']

In [ ]:
len(documents)

## Get document embeddings

In [ ]:
document_embeddings = model.encode(documents)

In [ ]:
document_embeddings.shape

In [ ]:
document_embeddings, document_embeddings.shape

In [ ]:
documents[0]

In [ ]:
document_embeddings[0].shape

In [ ]:
document_embeddings[0]

## Let's try to find the most similar document for one query

### New Query

In [ ]:
new_text = 'What is AI?'
new_text

### Get Embedding for New Query

In [ ]:
new_text_embedding = model.encode(new_text)
new_text_embedding.shape

### Get Cosine Similarity Score of Document Emebddings compared to New Query Embedding

In [ ]:
cos_scores = util.pytorch_cos_sim(new_text_embedding, document_embeddings)[0]
cos_scores

### Get Most Similar Document ID

In [ ]:
top_results = torch.topk(cos_scores, k=1)
top_results

In [ ]:
idx = top_results.indices[0]
idx

### Get Most Similar Document

In [ ]:
documents[idx]

## Create a function to return the top similar document based on any query

__Replace `<REPLACE WITH CODE HERE>` sections with your own code__

In [ ]:
def semantic_search_engine(query, embedder_model):
 <REPLACE WITH CODE HERE>

## Try out the function

In [ ]:
new_sentence = 'Tell me about AI'
semantic_search_engine(new_sentence, model)

In [ ]:
new_sentence = 'Do you know about the pyramids?'
semantic_search_engine(new_sentence, model)

In [ ]:
new_sentence = 'How do plants survive?'
semantic_search_engine(new_sentence, model)

In [ ]:
new_sentence = 'How do animals survive?'
semantic_search_engine(new_sentence, model)